## Deliverable 2. Create a Customer Travel Destinations Map.

In [36]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,City_Descr
0,0,Bredasdorp,ZA,2020-08-18 20:34:54,-34.53,20.04,50.00,76,76,5.82,broken clouds
1,1,Avarua,CK,2020-08-18 20:31:31,-21.21,-159.78,73.40,73,83,17.22,light rain
2,2,Cockburn Town,TC,2020-08-18 20:34:55,21.46,-71.14,87.80,70,40,9.17,scattered clouds
3,3,Rikitea,PF,2020-08-18 20:34:55,-23.12,-134.97,71.96,79,0,10.63,clear sky
4,4,Bambous Virieux,MU,2020-08-18 20:34:56,-20.34,57.76,72.00,73,20,10.29,few clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,City_Descr
2,2,Cockburn Town,TC,2020-08-18 20:34:55,21.46,-71.14,87.80,70,40,9.17,scattered clouds
5,5,Tazmalt,DZ,2020-08-18 20:34:56,36.39,4.40,77.00,78,99,2.24,overcast clouds
6,6,Diffa,NE,2020-08-18 20:34:57,13.32,12.61,83.62,57,3,6.13,clear sky
10,10,Labuhan,ID,2020-08-18 20:34:58,-6.88,112.21,75.60,80,43,7.87,scattered clouds
19,19,Eydhafushi,MV,2020-08-18 20:35:03,5.10,73.07,82.22,67,1,2.57,clear sky
20,20,Hithadhoo,MV,2020-08-18 20:35:03,-0.60,73.08,83.73,76,0,10.47,clear sky
22,22,Hilo,US,2020-08-18 20:34:51,19.73,-155.09,80.60,61,50,8.05,light rain
25,25,Ribeira Grande,PT,2020-08-18 20:35:28,38.52,-28.70,75.20,94,75,26.40,broken clouds
30,30,Puerto Ayora,EC,2020-08-18 20:35:45,-0.74,-90.35,77.00,65,40,24.16,scattered clouds
31,31,Gharghoda,IN,2020-08-18 20:35:48,22.17,83.35,75.97,94,100,3.27,light rain


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID       221
City          221
Country       221
Date          221
Lat           221
Lng           221
Max Temp      221
Humidity      221
Cloudiness    221
Wind Speed    221
City_Descr    221
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "City_Descr", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,City_Descr,Lat,Lng,Hotel Name
2,Cockburn Town,TC,87.80,scattered clouds,21.46,-71.14,
5,Tazmalt,DZ,77.00,overcast clouds,36.39,4.40,
6,Diffa,NE,83.62,clear sky,13.32,12.61,
10,Labuhan,ID,75.60,scattered clouds,-6.88,112.21,
19,Eydhafushi,MV,82.22,clear sky,5.10,73.07,
20,Hithadhoo,MV,83.73,clear sky,-0.60,73.08,
22,Hilo,US,80.60,light rain,19.73,-155.09,
25,Ribeira Grande,PT,75.20,broken clouds,38.52,-28.70,
30,Puerto Ayora,EC,77.00,scattered clouds,-0.74,-90.35,
31,Gharghoda,IN,75.97,light rain,22.17,83.35,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}


In [10]:
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [23]:
# check how many blank hotel rows exist
print(hotel_df.count())
count = hotel_df.loc[(hotel_df["Hotel Name"] == "")].count()
print(count)

City          221
Country       221
Max Temp      221
City_Descr    221
Lat           221
Lng           221
Hotel Name    221
dtype: int64
City          19
Country       19
Max Temp      19
City_Descr    19
Lat           19
Lng           19
Hotel Name    19
dtype: int64


In [48]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].replace('', np.nan, inplace=True)


In [50]:
clean_hotel_df = hotel_df
clean_hotel_df

,City,Country,Max Temp,City_Descr,Lat,Lng,Hotel Name
2,Cockburn Town,TC,87.80,scattered clouds,21.46,-71.14,Osprey Beach Hotel
5,Tazmalt,DZ,77.00,overcast clouds,36.39,4.40,Hôtel Numédia
6,Diffa,NE,83.62,clear sky,13.32,12.61,ProDAF/Diffa
10,Labuhan,ID,75.60,scattered clouds,-6.88,112.21,bador sepret
19,Eydhafushi,MV,82.22,clear sky,5.10,73.07,Soneva Fushi
...,...,...,...,...,...,...,...
650,Trincomalee,LK,80.04,overcast clouds,8.57,81.23,Welcombe Hotel
655,Gigmoto,PH,79.57,broken clouds,13.78,124.39,Cj's Travellers Inn
659,Awbari,LY,88.61,clear sky,26.59,12.78,فندق أوباري
664,Grafton,US,82.99,scattered clouds,42.21,-71.69,The Grafton Inn


In [52]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [56]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>Weather Description</dt><dd>{City_Descr}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [57]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Call the figure to plot the data.
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))